<a href="https://colab.research.google.com/github/appletreeleaf/Projects/blob/main/Viterbi_Algorithm_for_pos_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Viterbi(object):
  def __init__(self, datas):
      emission, transition, input_text = [], [], []
      trans_row, em_row, input_row = 0, 0, 0

      # 입력에서 정수부분 추출
      for data in datas:
        if len(data) == 1:
            if trans_row == 0:
                trans_row = int(data[0])
            elif em_row == 0:
                em_row = int(data[0])
            elif input_row == 0:
                input_row = int(data[0])

      print(trans_row, em_row, input_row)
      emission  = datas[trans_row+2:trans_row+2+em_row]
      print('emission: ', emission)
      transition = datas[1:trans_row+1]
      print('transition: ', transition)
      input_text = datas[-input_row:]
      input_text = [data[0].split() for data in input_text]
      print('input_text: ', input_text)

      # 2-dim matrix
      trans_matrix = [l[:2] for l in transition]
      print("trans_matrix", trans_matrix)

      states = list(set(sum(trans_matrix, [])))
      # 스페셜 토큰 제거
      del(states[states.index('<s>')])
      del(states[states.index('</s>')])
      self.states = states




      # 단어 발생 확률 체크
      dict_em = {}
      for row in emission:
          try:
              dict_em[row[0]]
          except:
              dict_em[row[0]] = {row[1] : float(row[2])}
          else:
              dict_em[row[0]][row[1]] = float(row[2])
      print("emission_dictionary: ", dict_em)
      self.dict_em = dict_em

      dict_trans = {}
      for row in transition:
          try:
              dict_trans[row[0]]
          except:
              dict_trans[row[0]] = {row[1] : float(row[2])}
          else:
              dict_trans[row[0]][row[1]] = float(row[2])

      self.dict_trans = dict_trans
      self.states = tuple(self.states)
      self.input_row = input_row
      self.trans_row = trans_row
      self.em_row = em_row
      self.input_text = input_text
      self.transition = transition
      self.emission = emission

  def decode(self):

      delta = [{}]
      path = {}
      for char in self.input_text:
          i = 0
          # start_prob
          for data in self.dict_trans['<s>']:
              delta[0][data] = self.dict_trans['<s>'][data]
              path[data] = data
          del(self.dict_trans["<s>"])

          for t,node in enumerate(char):
              t = t+1
              if len(char) <= t:
                  break

              delta.append({})
              new_path = {}

              # calculate
              for y in self.states:
                (probability, where) = max((delta[t-1][da] * self.dict_trans[da][y] * self.dict_em[y][char[t]], da) for da in self.states)
                delta[t][y] = probability

                try:
                    new_path[y] = (path[where]) + [y]
                except:
                    new_path[y] = [(path[where])] + [y]

              path = new_path
          t -= 1
          (probability, where) = max((delta[t][y], y) for y in self.states)
          return (path[where]) #[self.states])

In [ ]:
import sys
from collections import defaultdict

#############
# functions #
#############

def eval(output, gold):
    """
    *** 수정하지 마시오
    테스트 평가 함수 (테스트 결과를 체크할수 있는 함수입니다.)
    :param output:      result of tagging(list of str)
    :param gold:        reference of input data(gold data)
    :return accuracy:   accuracy of test
    """
    golds = [_.split() for _ in gold]
    outs = [_.split() for _ in output]
    if len(golds) != len(outs):
        print('numbers of sentences are different %d vs %d' % (len(golds), len(outs)), file=sys.stderr)
        return False
    correct_num = 0
    for line_num, (gold, out) in enumerate(zip(golds, outs), start=1):
        if len(gold) != len(out):
            print('numbers of tokens in %d-th sentence are different %d vs %d' % \
                   (line_num, len(gold), len(out)), file=sys.stderr)
            return False
        correct_num += sum([(1 if g == o else 0) for g, o in zip(gold, out)])
    total_num = sum([len(_) for _ in golds])
    accuracy = 100.0 * correct_num / total_num
    print('Accuracy: %.1f%%' % accuracy)
    return accuracy



########
# main #
########
def main(inp, ref):
	seqs = ''
	with open(inp, 'rt') as inp_:
		for seq in inp_:
			seqs += seq

	seqs = seqs.split("\n")

	# 띄어쓰기 기준으로 분리된 sequence들을 tab키 기준으로 또 분리시킴
	datas = ([seq.split("\t") for seq in seqs])
	viterbi = Viterbi(datas)
	print(' '.join(viterbi.decode()))
	# result = [' '.join(viterbi.decode())]

	refs = []
	with open(ref, 'rt') as rh:
		for text in rh.readlines():
			refs.append(text.strip())

	# if len(result) == len(refs):
	# 		eval(result, refs)
	# return True

if __name__ == '__main__':
    # input_file : 입력 텍스트 파일
    # ref_file : 레퍼런스 텍스트 파일, 결과값과 비교하여 정확도를 계산하는데 이용합니다.
    input_file = '/content/drive/MyDrive/Colab Notebooks/Data/input.txt'
    ref_file = "/content/drive/MyDrive/Colab Notebooks/Data/ref.txt"

    main(input_file, ref_file)

9 4 1
emission:  [['A', 'x', '0.4'], ['A', 'y', '0.6'], ['B', 'x', '0.3'], ['B', 'y', '0.7']]
transition:  [['<s>', 'A', '0.7'], ['<s>', 'B', '0.3'], ['<s>', '</s>', '0'], ['A', 'A', '0.2'], ['A', 'B', '0.7'], ['A', '</s>', '0.1'], ['B', 'A', '0.7'], ['B', 'B', '0.2'], ['B', '</s>', '0.1']]
input_text:  [['x', 'y', 'y', 'x']]
trans_matrix [['<s>', 'A'], ['<s>', 'B'], ['<s>', '</s>'], ['A', 'A'], ['A', 'B'], ['A', '</s>'], ['B', 'A'], ['B', 'B'], ['B', '</s>']]
emission_dictionary:  {'A': {'x': 0.4, 'y': 0.6}, 'B': {'x': 0.3, 'y': 0.7}}
A B A B


In [ ]:
refs = []
with open(ref_file, 'rt') as rh:
		for text in rh.readlines():
			refs.append(text.strip())
print(refs)

['A B A']
